In [36]:
import os
import cv2 as cv
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [37]:
dstdir = "./raw_data/"

try:
    os.makedirs(dstdir)
except FileExistsError:
    pass

In [2]:
""" Correr una sola vez para descomprimir las imagenes
en el folder deseado. Cambiar la ruta al archivo .zip
de ser necesario """

!unzip -qq "../plant-pathology-2020-fgvc7.zip" -d dstdir

In [38]:
data = pd.read_csv(dstdir + "train.csv")

In [39]:
sizes = list(data.iloc[:, 1:].sum())
labels = data.columns
# print(sizes)

fig = go.Figure(data=[go.Pie(labels=labels, values=sizes)])
fig.show()

In [40]:
def resize_img(image):

    # width = height = 224
    scale = 0.4 # porcentaje del tamano original (40%)
    width = int(image.shape[1] * scale)
    height = int(image.shape[0] * scale)
    dim = (width, height)

    return cv.resize(image, dim, interpolation = cv.INTER_AREA)

In [41]:
def preprocess(image_path):
    
    bgr_image = cv.imread(image_path)
    bgr_image_r = resize_img(bgr_image)
    rgb_image = cv.cvtColor(bgr_image_r, cv.COLOR_BGR2RGB)

    #Preprocessing
    
    gray_image = cv.cvtColor(rgb_image, cv.COLOR_RGB2GRAY)
    blur_image = cv.GaussianBlur(gray_image, (25,25), 0)
    _, bw_otsu_image = cv.threshold(blur_image, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)
    kernel = np.ones((25,25), np.uint8)
    closing_image = cv.morphologyEx(bw_otsu_image, cv.MORPH_CLOSE, kernel)
    list = [bgr_image_r, rgb_image, gray_image, blur_image, bw_otsu_image, closing_image]
    
    return list


La funcion `data_processing()`, hasta el momento crea la mascara para eliminar el fondo

In [42]:
def data_processing(images_folder_path):

    train_images = []
    test_images = []

    images_list = [images_folder_path + image for image in os.listdir(images_folder_path)]

    for img_path in images_list:
        image = preprocess(img_path)
        train_images.append(image) if img_path.startswith(images_folder_path + "Train") else test_images.append(image)
    
    # return train_images, test_images

In [49]:
def save_preprocessing_samples(sample_image_path):

    directory = './step_by_step/'

    try:
        os.makedirs(directory)
    except FileExistsError:
        pass

    samples = preprocess(sample_image_path)
    names = ['1. bgr_image.png', '2. rgb_image.png', '3. gray_image.png', '4. blur_image.png', '5. otsu_image.png', '6. closing_image.png']
    
    for title, image in zip(names, samples):
        cv.imwrite(directory + title, image)
    

In [44]:
path = dstdir + "images/"
# data_processing(path)

In [51]:
sample_image_path = "./sample_image/sample1.jpg"
save_preprocessing_samples(sample_image_path)

In [ ]:
# fig, ax = plt.subplots(1, 7, figsize=(20, 20))
# for i in range(7):
#     ax[i].set_axis_off()
#     ax[i].imshow(train[3][i])


# cv.imshow("Mostrar Imagen", train[0][3])
# cv.waitKey(0) 
# cv.destroyAllWindows()